In [1]:
pip install boto3

  Obtaining dependency information for boto3 from https://files.pythonhosted.org/packages/01/c8/fcf1ec25b0320af58be3a3d8c0f8ed06513133c65b318895e7eb39ef14ab/boto3-1.35.66-py3-none-any.whl.metadata
  Obtaining dependency information for botocore<1.36.0,>=1.35.66 from https://files.pythonhosted.org/packages/92/f0/372c2474cd198485ac427ccdd54796b05c2e730bdff0523c26012fe40ad7/botocore-1.35.66-py3-none-any.whl.metadata
  Obtaining dependency information for s3transfer<0.11.0,>=0.10.0 from https://files.pythonhosted.org/packages/66/05/7957af15543b8c9799209506df4660cba7afc4cf94bfb60513827e96bed6/s3transfer-0.10.4-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 36.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 11.7 MB/s eta 0:00:00
  Attempting uninstall: botocore
    Found existing installation: botocore 1.29.76
    Uninstalling botocore-1.29

In [1]:
pip install requests beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [17]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import boto3
import logging

In [21]:
class WebExtPipeline:
    def __init__(self,url):
        self.url = url
        #Setup logging
        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger(__name__)
    
    def extract_data(self,selector:str=None)->list[dict]:
        try:
            response = requests.get(self.url)
            response.raise_for_status()

            soup = BeautifulSoup(response.text,'html.parser')
            if selector:
                elements = soup.select(selector)
                data = [self._parse_element(el) for el in elements]
            else:
                data = self._generic_extract(soup)
                
            print(data)
            return data

        except requests.RequestException as e:
            self.logger.error(f"Scraping error:{e}")
            return[]
    
    def _parse_element(self, element) -> dict:
        return {
            'text': element.get_text(strip=True), 
            'href': element.get('href','')
        }
    def _generic_extract(self,soup)->list[dict]:
        return [{'text': p.get_text(strip=True)} for p in soup.find_all('p')]
    
    def run_pipeline(self, selector: str = None):
        data = self.extract_data(selector)

In [23]:
def main():
    pipeline = WebExtPipeline('https://www.basketball-reference.com/players/')
    pipeline.run_pipeline(selector='div')

if __name__ == '__main__':
    main()

[{'text': 'Sports\xa0Reference\u202f®BaseballFootball(college)Basketball(college)HockeyFootballBlogStathead\u202f®Immaculate Grid\u202f®Questions or Comments?Welcome·Your AccountLogoutAd-Free LoginCreate AccountMENUPlayersTeamsSeasonsLeadersScoresWNBADraftStatheadNewsletterFull Site Menu BelowYou are here:BBR Home Page>PlayersWelcome·Your AccountLogoutAd-Free LoginCreate AccountList of all the NBA and ABA PlayersIndex of LettersActive players are listed inbold.* Indicates member of the Hall of Fame.AKareem Abdul-Jabbar*,Ray Allen*,LaMarcus Aldridge,Giannis Antetokounmpo,Paul Arizin*,Carmelo Anthony,\xa0…BCharles Barkley*,Kobe Bryant*,Larry Bird*,Walt Bellamy*,Rick Barry*,Chauncey Billups*,\xa0…CWilt Chamberlain*,Stephen Curry,Vince Carter*,Maurice Cheeks*,Tyson Chandler,Mike Conley,\xa0…DTim Duncan*,Kevin Durant,Clyde Drexler*,Adrian Dantley*,Anthony Davis,DeMar DeRozan,\xa0…EJulius Erving*,Patrick Ewing*,Alex English*,Dale Ellis,Joel Embiid,James Edwards,\xa0…FWalt Frazier*,Michael Fi